In [20]:
from pywikiapi import Site
from bs4 import BeautifulSoup
import requests
import spacy

from link_research import recup_links


In [21]:
source = requests.get('https://en.wikipedia.org/wiki/Wikipedia:Most-wanted_articles').text

soup = BeautifulSoup(source, 'lxml')



content = soup.find('div', class_='mw-content-ltr')

liste_new = []
liste_link = []
liste_noms = []

for new in content.find_all('a', class_='new'):
    liste_new.append(new)

nlp = spacy.load("en_core_web_sm")
   
    
for new in liste_new:
    doc = nlp(new.text)
    for ent in doc.ents:
        #print(ent.text, ent.start_char, ent.end_char, ent.label_)
        if ent.label_ == 'PERSON':
            liste_noms.append(new)
            

In [25]:
#enlever de la liste les entrées contenant des numéros
for nom in liste_noms:
    i = 0
    for i in nom.text:
        if i.isdigit():
            if nom in liste_noms :
                liste_noms.remove(nom)
                
            
#passage par un dictionnaire pour enlever d'éventuels doublons dans la liste
liste_noms = list(dict.fromkeys(liste_noms))


In [23]:
for nom in liste_noms:
    words = []
    words.append(nom.text.split())
    for item in words:
        for word in item:
            doc = nlp(word)
            for ent in doc.ents:
                #print(ent.text, ent.start_char, ent.end_char, ent.label_)
                if ent.label_ != 'PERSON':
                    if nom in liste_noms:
                        liste_noms.remove(nom)

In [26]:
for nom in liste_noms:
    print(nom.text)

Terry Vaughn (referee)
Guyora Binder
Torneo Federal B
Leigh Armstrong
Graham Salisbury (referee)
Dean Whitestone
Ng Wing Yung
Lei On Kei
Greg Aitken
Anagennisi Epanomi F.C.
Nigel Miller
Ng Kai Lam
Quin Blackburn
Geoff Eltringham
Charles G. Waugh
Jeremy Simpson
William Austin (film editor)
Yeung Nga Ting
Hiroyuki Kimura (referee)
Nikolai Vasilyevich Parin
Francesca Stephenson
Carl Boyeson
Darren Bond
Andy Woolmer
Tom Appelman
Chad Patton
FC Wien
James Linington
Mark Heywood
Armando Villarreal
Ivan Clementi
G.D. Blodgett
Chiaki Okadaue
Marco Gentile (motorcyclist)
Alena Tarasova
Charles Breakspear
Eric Bataille
Stephen Schubert
Avalyn Hunter
Ruben Hernandez (jockey)
Alan Extreme
Neal Brizan
Eddie Ilderton
Scott Hanna
Tony Harrington
Gustav Reiner
Joshua Bondo
Nicholas Schmassman
Barbara Schwarz
Conor O'Dwyer
Michael Orr
Andy Jao
Joe Wu
Josef Doppler (motorcyclist)
Jason Vincent (motorcyclist)
Tan Chi Yan
John H. Roscoe
Aziz Asimov
Janet Murphy
Ross Joyce
Yu Ming-hsun
Johan Verbist
Luis S

In [18]:
user = "YGrebici@MostWantedBot"
passw = "rebp70qighbgk55lpoi47b8fnesf3au3"
baseurl = 'http://wikipast.epfl.ch/wikipast/'
summary = 'MostWantedBot page creation'

baseurl = 'http://wikipast.epfl.ch/wikipast/'
baseurl + 'api.php'

site = Site('http://wikipast.epfl.ch/wikipast/api.php')  # Définition de l'adresse de l'API
site.no_ssl = True  # Désactivation du https, car non activé sur wikipast
site.login(user, passw)  # Login du bot

In [19]:
for i in range(0,2):
    results = []
    MostWantedTerm = liste_noms[i].text
    print(MostWantedTerm)
    # Cherche dans toutes les pages celles commençant par Mahatma
    for res in site.query(list='allpages', apprefix=MostWantedTerm):
        results.append(res)

    # check if page already exists
    if len(results[0]['allpages']) == 1:
        print('Page déjà existante')

    else:
        # Login request
        payload = {'action': 'query', 'format': 'json', 'utf8': '', 'meta': 'tokens', 'type': 'login'}
        r1 = requests.post(baseurl + 'api.php', data=payload)

        # login confirm
        login_token = r1.json()['query']['tokens']['logintoken']
        payload = {'action': 'login', 'format': 'json', 'utf8': '', 'lgname': user, 'lgpassword': passw,
                   'lgtoken': login_token}
        r2 = requests.post(baseurl + 'api.php', data=payload, cookies=r1.cookies)

        # get edit token2  comme si on appuyait sur le bouton modifier?
        # le csrf token est un jeton de confirmation de sécurité
        params3 = '?format=json&action=query&meta=tokens&continue='
        r3 = requests.get(baseurl + 'api.php' + params3, cookies=r2.cookies)
        edit_token = r3.json()['query']['tokens']['csrftoken']

        edit_cookie = r2.cookies.copy()
        edit_cookie.update(r3.cookies)

        content_tab = recup_links(MostWantedTerm)

        content = ''

        # contenu de la page (vide)
        for element in content_tab:
            content += element

        # save action (modif de la page avec l'action edit)
        payload = {'action': 'edit', 'assert': 'user', 'format': 'json', 'utf8': '', 'text': content, 'summary': summary,
                   'title': MostWantedTerm, 'token': edit_token}
        r4 = requests.post(baseurl + 'api.php', data=payload, cookies=edit_cookie)

        # payload semble etre une fonction qui va créer un url avec des commandes: ici fonction principale est edit, puis on y ajoute les arguments

        r4.json()

        # dans le r4 (requete 4 je pense), mis sous format json, on a le result:succes donc page MostWantedTerm bien créée

Anagennisi Giannitsa F.C.
Terry Vaughn (referee)
